In [1]:
%matplotlib qt5
import mne
from mne.preprocessing import ICA
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import os
import os.path as op
from autoreject import AutoReject

from mne.connectivity import spectral_connectivity
from mne.viz import plot_sensors_connectivity
from mne.viz import ClickableImage  # noqa
from mne.viz import (plot_alignment, snapshot_brain_montage,
                     set_3d_view)

import bct
import pickle

from mpl_toolkits import mplot3d

## FEATURES OF COH AND PLV

In [23]:
#Excombatants
load_path = ('IAPS crudos/Casos/Medida 1/01_Preprocessed/')
path_save= ('IAPS crudos/Casos/Medida 1/02_Features/')
#Controls
# load_path = ('IAPS crudos/Controles/01_Preprocessed/')
# path_save= ('IAPS crudos/Controles/02_Features/')

EEG_Files = os.listdir(load_path)
epochs_name= ['Negative','Neutral','Positive']

#Loading Subject's Epochs Files
thresholds = [0.3,0.4,0.5,0.6,0.7]
for epoch_ind in range(len(epochs_name)):
    Features=np.zeros([len(EEG_Files),5*len(thresholds),5])
    for file in range(0,len(EEG_Files)):
        print(epoch_ind)
        _subject= file
        print('ANALIZING SUBJECT %02d' %(file))
        file_exist = 0
        try:
            filename = (load_path+EEG_Files[_subject])
            epochs = mne.read_epochs(filename).pick_types(eeg=True,eog = False)  
            epochs_r = epochs.copy().resample(512, npad='auto')
            print(epochs_r.get_data().shape)
            file_exist = 1
            print(filename)

        except:
            print('ERROR:',filename + ' not found')

    #Setting the epochs by IAPS Valence
        epochs_neg = epochs_r["10"]
        epochs_neu = epochs_r["11"]
        epochs_pos = epochs_r["12"]

    #Vectorizing IAPS epochs
        total_epochs= [epochs_neg,epochs_neu,epochs_pos]
        epochs = total_epochs[epoch_ind]

        filename_save= (path_save+EEG_Files[_subject])
    #Calculating the connectivity using coherence and multitaper, also frquency bands are defined        
        data=epochs.get_data()
        picks = mne.pick_types(epochs.info,eeg = True,eog = False)
        sfreq=512
        con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
            data, method='coh', mode='multitaper', sfreq=sfreq, fmin=(0., 4., 8., 16., 32.),
                                                                     fmax=(4., 8., 16., 32., 64.),
            faverage=True, tmin=0, mt_adaptive=False, n_jobs=1)

    #Vars to store the features and thresholds used
        #thresholds = [0.3,0.4,0.5,0.6,0.7] #coherence and plv
        
        Nband= len(con[1][1])
        Nth= len(thresholds)
        L_f = np.zeros([len(thresholds),Nband])
        eff_f = np.zeros([len(thresholds),Nband]) 
        C_f = np.zeros([len(thresholds),Nband])
        El_f = np.zeros([len(thresholds),Nband])
        S_f = np.zeros([len(thresholds),Nband])
        th_ind=0
        index=0



    #Loop start
        for th in thresholds:
            con_gt= np.zeros([len(con[1][1]),len(con[:,0,0]),len(con[:,0,0])])
            Gr= np.zeros_like(con_gt)
            #print(Gr.shape,con.shape)

            for i in range(0,len(con[1,1])):
                con_gt[i] = con[:,:,i] + con[:,:,i].T + np.identity(62)
                Gr[i] = np.zeros((len(con[:,0,0]),62))
                ind = np.where(con_gt[i]>th)
                Gr[i][ind] = 1

            Gr=np.swapaxes(Gr,0,2)
            #print(Gr.shape,con.shape)

        #For every frequency bands features are calculated
            for nb in range(0,len(Gr[0,0,:])): #Frequency bands
                #Characteristic path length of the network
                #G = squeeze(Gr(ntr,nth,:,:,nb));
                D = bct.distance_bin(Gr[:,:,nb])
                [L,eff,_,_,_] = bct.charpath(D,include_infinite=False);
                #Clustering coefficient of the network
                Ci= bct.clustering_coef_bu(Gr[:,:,nb])
                C = np.mean(Ci)
                # Local efficiency of the network
                Eloc= bct.efficiency_bin(Gr[:,:,nb],local=False);
                El = np.mean(Eloc)
                # Measure `of network small-worldness.

                try:
                    Grand = bct.randomizer_bin_und(Gr[:,:,nb],1);
                    Drand = bct.distance_bin(Grand);
                    Lrand = bct.charpath(Drand,include_infinite=False);
                    Lrand = Lrand[0]
                    Cirand= bct.clustering_coef_bu(Gr[:,:,nb])
                    Crand= np.mean(Cirand);
                    S=(C/Crand)/(L/Lrand);

                except:
                    S=0
                    #print('NO SE CARGO S')



            #Storing the features calculated for threshold and band in a matrix for each feature
                #Features[index]
                #L_f[th_ind,nb]= L
                #eff_f[th_ind,nb]= eff
                #C_f[th_ind,nb]= C
                #El_f[th_ind,nb]= El
                #S_f[th_ind,nb]= S
                index= Nband*(th_ind)+nb
                #print(index,Nband,th_ind,nb)
                Features[_subject,index]= [L, eff, C, El, S]
                #print(Features)


            th_ind+=1


    #Features vector



#Saving data
    print(Features.shape)
    filename_save= filename_save[:-7] 
    filename_save= filename_save+'coh_'+epochs_name[epoch_ind]
    file = open(filename_save, 'wb')
    # dump information to that file
    pickle.dump(Features, file)
    # close the file
    file.close()



0
ANALIZING SUBJECT 00
Reading IAPS crudos/Casos/Medida 1/01_Preprocessed/PE_I001_IAPS_epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
196 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
(196, 62, 513)
IAPS crudos/Casos/Medida 1/01_Preprocessed/PE_I001_IAPS_epo.fif
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 1891 connections
    using t=0.000s..1.000s for estimation (513 points)
    computing connectivity for the bands:
     band 1: 0.0Hz..4.0Hz (5 points)
     band 2: 5.0Hz..8.0Hz (4 points)
     band 3: 9.0Hz..16.0Hz (8 points)
     band 4: 17.0Hz..31.9Hz (16 points)
     band 5: 32.9Hz..63.9Hz (32 points)
    connec

C:\Users\alexi\anaconda3\lib\site-packages\mne\connectivity\spectral.py:1010: RuntimeWarning: divide by zero encountered in double_scalars
  5. / np.min(fmin), five_cycle_freq))
<ipython-input-23-fcabb67bbf59>:45: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.002 sec, need at least inf sec epochs or fmin=4.990. Spectrum estimate will be unreliable.
  con, freqs, times, n_epochs, n_tapers = spectral_connectivity(


    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    

C:\Users\alexi\anaconda3\lib\site-packages\mne\connectivity\spectral.py:1010: RuntimeWarning: divide by zero encountered in double_scalars
  5. / np.min(fmin), five_cycle_freq))
<ipython-input-23-fcabb67bbf59>:45: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.002 sec, need at least inf sec epochs or fmin=4.990. Spectrum estimate will be unreliable.
  con, freqs, times, n_epochs, n_tapers = spectral_connectivity(


    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
   

...

    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
   

C:\Users\alexi\anaconda3\lib\site-packages\mne\connectivity\spectral.py:1010: RuntimeWarning: divide by zero encountered in double_scalars
  5. / np.min(fmin), five_cycle_freq))
<ipython-input-23-fcabb67bbf59>:45: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.002 sec, need at least inf sec epochs or fmin=4.990. Spectrum estimate will be unreliable.
  con, freqs, times, n_epochs, n_tapers = spectral_connectivity(


    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
   

C:\Users\alexi\anaconda3\lib\site-packages\mne\connectivity\spectral.py:1010: RuntimeWarning: divide by zero encountered in double_scalars
  5. / np.min(fmin), five_cycle_freq))
<ipython-input-23-fcabb67bbf59>:45: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.002 sec, need at least inf sec epochs or fmin=4.990. Spectrum estimate will be unreliable.
  con, freqs, times, n_epochs, n_tapers = spectral_connectivity(


    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
   

In [18]:
print(Features.shape)
print(Features[0,20])

(19, 30, 5)
[1.8351841  0.66233197 0.83258295 0.59928609 0.89299784]


## FEATURES OF IMCOH 

In [ ]:
load_path = ('IAPS crudos/Controles/01_Preprocessed/')
path_save= ('IAPS crudos/Controles/02_Features/')
EEG_Files = os.listdir(load_path)
epochs_name= ['Negative','Neutral','Positive']

#Loading Subject's Epochs Files

for epoch_ind in range (len(epochs_name)):
    Features=np.zeros([len(EEG_Files),6,1891])
    for file in range (0,len(EEG_Files)):
        print(epoch_ind)
        _subject= file
        print('ANALIZING SUBJECT %02d' %(file))
        file_exist = 0
        try:
            filename = (load_path+EEG_Files[_subject])
            epochs = mne.read_epochs(filename).pick_types(eeg=True,eog = False)  
            epochs_r = epochs.copy().resample(512, npad='auto')
            print(epochs_r.get_data().shape)
            file_exist = 1
            print(filename)

        except:
            print('ERROR:',filename + ' not found')

    #Setting the epochs by IAPS Valence
        epochs_neg = epochs_r["10"]
        epochs_neu = epochs_r["11"]
        epochs_pos = epochs_r["12"]

    #Vectorizing IAPS epochs
        total_epochs= [epochs_neg,epochs_neu,epochs_pos]
        epochs = total_epochs[epoch_ind]

        filename_save= (path_save+EEG_Files[_subject])
    #Calculating the connectivity using coherence and multitaper, also frquency bands are defined        
        data=epochs.get_data()
        picks = mne.pick_types(epochs.info,eeg = True,eog = False)
        sfreq=512
        con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
            data, method='plv', mode='multitaper', sfreq=sfreq, fmin=(0., 4., 8., 16., 32.),
                                                                     fmax=(4., 8., 16., 32., 64.),
            faverage=True, tmin=0, mt_adaptive=False, n_jobs=1)

        con= np.swapaxes(con,0,2)
        a=np.zeros([6,1891])
        for bands in range(0,6):
            contador=0
            for i in range(len(con[0,0])-1):
                for j in range(i+1,len(con[0,0])):
                    #print(con[0,i,j])
                    a[bands,(contador)] = con[bands,i,j]
                    contador+=1

    #Features vector
        Features[_subject] = a
        print(Features.shape)


#Saving data
    #Features = np.swapaxes(Features,1,2)
    filename_save= filename_save[:-7] 
    filename_save= filename_save+'coh_'+epochs_name[epoch_ind]
    file = open(filename_save, 'wb')
    # dump information to that file
    pickle.dump(Features, file)
    # close the file
    file.close()



In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
con= np.swapaxes(con,0,2)
print(len(con[0,0]))
a=np.zeros([6,1891])

contador=0
for i in range(len(con[0,0])-1):
    for j in range(i+1,len(con[0,0])):
        #print(con[0,i,j])
        a[0,(contador)] = con[0,i,j]
        contador+=1
    
a[0]

    
    
    



In [ ]:
Features[0,2] == a[2]


In [3]:
#print(Features.shape)
path='IAPS crudos/Casos/Medida 1/01_Preprocessed/PE_I030_IAPS_epo.fif'
epochs = mne.read_epochs(path).pick_types(eeg=True,eog = False) 
evoked=epochs.average()
print(evoked)
epochs_r = epochs.copy().resample(100, npad='auto')

Reading IAPS crudos/Casos/Medida 1/01_Preprocessed/PE_I030_IAPS_epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 62) active
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
236 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
Created an SSP operator (subspace dimension = 1)
1 projection items activated
<Evoked  |  '0.33 * 10 + 0.33 * 11 + 0.33 * 12' (average, N=236), [-0.2, 0.8] sec, 62 ch, ~612 kB>


In [75]:
from scipy import linalg
epochs_neg = epochs_r["11"]
data=epochs_neg.get_data()
print(data.shape)
picks = mne.pick_types(epochs_neg.info,eeg = True,eog = False)
sfreq=100
con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
    epochs, method='coh', mode='multitaper', sfreq=sfreq, fmin=(0., 4., 8., 12., 16., 32.),
                                                             fmax=(4., 8., 12., 16., 32., 50.),
    faverage=True, tmin=0, mt_adaptive=False, n_jobs=1)

#plot_sensors_connectivity(epochs_neg.info, con[:, :, 4])

th=0.7

print(con.shape)
Gr= np.zeros_like(con)
ind = np.where(con>th)
Gr[ind] = 1
print(Gr[:,:,5])
#plot_sensors_connectivity(epochs_neg.info, Gr[:,:,5])

con_a = con[:,:,5] + con[:,:,5].T + np.identity(62)
Gr_a= np.zeros_like(con_a)
ind = np.where(con_a>th)
Gr_a[ind] = 1

plt.figure()
plt.imshow(Gr_a[:,:])
plt.colorbar()
plt.show()




(79, 62, 100)
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 1891 connections
    using t=0.000s..1.000s for estimation (1001 points)
    computing connectivity for the bands:
     band 1: 0.0Hz..4.0Hz (5 points)
     band 2: 5.0Hz..8.0Hz (4 points)
     band 3: 9.0Hz..12.0Hz (4 points)
     band 4: 13.0Hz..16.0Hz (4 points)
     band 5: 17.0Hz..32.0Hz (16 points)
     band 6: 33.0Hz..50.0Hz (18 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6


<ipython-input-75-7f703341fa29>:10: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.001 sec, need at least inf sec epochs or fmin=4.995. Spectrum estimate will be unreliable.
  faverage=True, tmin=0, mt_adaptive=False, n_jobs=1)


    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
   

    computing connectivity for epoch 210
    computing connectivity for epoch 211
    computing connectivity for epoch 212
    computing connectivity for epoch 213
    computing connectivity for epoch 214
    computing connectivity for epoch 215
    computing connectivity for epoch 216
    computing connectivity for epoch 217
    computing connectivity for epoch 218
    computing connectivity for epoch 219
    computing connectivity for epoch 220
    computing connectivity for epoch 221
    computing connectivity for epoch 222
    computing connectivity for epoch 223
    computing connectivity for epoch 224
    computing connectivity for epoch 225
    computing connectivity for epoch 226
    computing connectivity for epoch 227
    computing connectivity for epoch 228
    computing connectivity for epoch 229
    computing connectivity for epoch 230
    computing connectivity for epoch 231
    computing connectivity for epoch 232
    computing connectivity for epoch 233
    computing co

In [52]:
con_a = con[:,:,3] + con[:,:,3].T + np.identity(62)
th = 0.4
Gr_a = np.zeros((62,62))
ind = np.where(con_a>th)
Gr_a[ind] = 1

#Features

#1. Characteristic path length of the network
D = bct.distance_bin(Gr_a)
[L,eff,_,_,_] = bct.charpath(D,include_infinite=False);
#Clustering coefficient of the network
Ci= bct.clustering_coef_bu(Gr_a);
C = np.mean(Ci);
# Local efficiency of the network
Eloc = bct.efficiency_bin(Gr_a,local=False);
El = np.mean(Eloc);
# Measure `of network small-worldness.
#try:
Grand = bct.randomizer_bin_und(Gr_a,1);
Drand = bct.distance_bin(Gr_a);
Lrand = bct.charpath(Drand,include_infinite=False);
Lrand = Lrand[0]
Cirand= bct.clustering_coef_bu(Gr_a);
Crand= np.mean(Cirand);
S=(C/Crand)/(L/Lrand);
print(L,eff,C,El,S)



2.875727128503437 0.43485935886781996 0.8150164822915247 0.43485935886781996 1.0


In [ ]:
print(EEG_Files)

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

th = 0.6
con_gt= np.zeros([len(con[1][1]),62,62])
Gr= np.zeros_like(con_gt)
print(Gr.shape,con.shape)

for i in range(0,len(con[1,1])):
    con_gt[i] = con[:,:,i] + con[:,:,i].T + np.identity(62)
    Gr[i] = np.zeros((62,62))
    ind = np.where(con_gt[i]>th)
    Gr[i][ind] = 1
    
Gr=np.swapaxes(Gr,0,2)
print(Gr.shape,con.shape)

for nb in range(0,len(Gr[0,0,:])): #Frequency bands
    #Characteristic path length of the network
    #G = squeeze(Gr(ntr,nth,:,:,nb));
    D = bct.distance_bin(Gr[:,:,nb])
    [L,eff,_,_,_] = bct.charpath(D,include_infinite=False);
    #Clustering coefficient of the network
    Ci= bct.clustering_coef_bu(Gr[:,:,nb])
    C = np.mean(Ci)
    # Local efficiency of the network
    Eloc= bct.efficiency_bin(Gr[:,:,nb],local=False);
    El = np.mean(Eloc)
    # Measure `of network small-worldness.
    #try:
    Grand = bct.randomizer_bin_und(Gr[:,:,nb],1);
    Drand = bct.distance_bin(Grand);
    Lrand = bct.charpath(Drand,include_infinite=False);
    Lrand = Lrand[0]
    Cirand= bct.clustering_coef_bu(Gr[:,:,nb])
    Crand= np.mean(Cirand);
    S=(C/Crand)/(L/Lrand);
    print(L,eff,C,El,S)
    


In [ ]:
Nfeats = 5;
print(len(Gr[0,0,:]))
for nb in range(0,len(Gr[0,0,:])): #Frequency bands
    #Characteristic path length of the network
    #G = squeeze(Gr(ntr,nth,:,:,nb));
    D = bct.distance_bin(Gr[:,:,nb])
    [L,eff,_,_,_] = bct.charpath(D,include_infinite=False);
    #Clustering coefficient of the network
    Ci= bct.clustering_coef_bu(Gr[:,:,nb])
    C = np.mean(Ci)
    # Local efficiency of the network
    Eloc= bct.efficiency_bin(Gr[:,:,nb],local=False);
    El = np.mean(Eloc)
    # Measure `of network small-worldness.
    #try:
    Grand = bct.randomizer_bin_und(Gr[:,:,nb],1);
    Drand = bct.distance_bin(Grand);
    Lrand = bct.charpath(Drand,include_infinite=False);
    Lrand = Lrand[0]
    Cirand= bct.clustering_coef_bu(Gr[:,:,nb])
    Crand= np.mean(Cirand);
    S=(C/Crand)/(L/Lrand);
    print(L,eff,C,El,S)
                
    #Concatenation
    #idx = Nb*(nth-1) + nb;
    #Feats(idx,ntr,:) = [L,eff,C,El,S];

In [ ]:

    [Ntr,Nth,Nc,Nc,Nb] = size(Gr);
    Nfeats = 5;
    
    Feats = zeros(Nb*Nth,Ntr,Nfeats);
    for ntr = 1:Ntr %trials
        for nth = 1:Nth %Thresholds
            for nb = 1:Nb %Frequency bands
                %Characteristic path length of the network
                G = squeeze(Gr(ntr,nth,:,:,nb));
                D = distance_bin(G);
                [L,eff,~,~,~] = charpath(D);
                %Clustering coefficient of the network
                Ci=clustering_coef_bu(G);
                C=mean(Ci);
                % Local efficiency of the network
                Eloc=efficiency_bin(G,1);
                El = mean(Eloc);
                % Measure `of network small-worldness.
                try
                    [Grand] = randomizer_bin_und(G,1);
                    Lrand = charpath(Grand);
                    Cirand=clustering_coef_bu(G);
                    Crand=mean(Cirand);
                    S=(C/Crand)/(L/Lrand);
                catch
                    S = 0;
                end
                %Concatenation
                idx = Nb*(nth-1) + nb;
                Feats(idx,ntr,:) = [L,eff,C,El,S];
            end
        end
    end

In [ ]:
print(Grand)